In [3]:
import urllib
import zipfile
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import numpy as np

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.4.1.50-1+cuda11.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
!unzip "/content/gdrive/MyDrive/NumpyLists/X_train.zip" -d "/content/"

Archive:  /content/gdrive/MyDrive/NumpyLists/X_train.zip
replace /content/X_train.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
X_test = np.load('/content/gdrive/MyDrive/NumpyLists/X_test.npz', allow_pickle=True)
y_test = np.load('y_test.npz', allow_pickle=True)

In [7]:
X_train = np.load('/content/gdrive/MyDrive/NumpyLists/X_val.npz', allow_pickle=True)
y_train = np.load('/content/gdrive/MyDrive/NumpyLists/y_val.npz', allow_pickle=True)


In [8]:
X_train = X_train['arr_0']
y_train = y_train['arr_0']

In [9]:
X_test = X_test['arr_0']
y_test = y_test['arr_0']

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4995, 150, 150, 3)
(4995,)
(4995, 150, 150, 3)
(4995,)


In [11]:
base_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = 'imagenet')


for layer in base_model.layers:
    layer.trainable = False

base_model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [12]:
last_layer = base_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [13]:
from tensorflow.keras.optimizers import RMSprop

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = tf.keras.layers.Dense  (1, activation='sigmoid')(x)           

model = tf.keras.Model(base_model.input, x) 

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [14]:
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [15]:
history = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_test, y_test),
                    steps_per_epoch = 100,
                    epochs = 20,
                    validation_steps = 50,
                    verbose = 2)

Epoch 1/20
100/100 - 26s - loss: 0.7553 - accuracy: 0.7237 - val_loss: 0.4222 - val_accuracy: 0.9856 - 26s/epoch - 261ms/step
Epoch 2/20
100/100 - 8s - loss: 0.3229 - accuracy: 0.9481 - val_loss: 0.1895 - val_accuracy: 1.0000 - 8s/epoch - 85ms/step
Epoch 3/20
100/100 - 10s - loss: 0.1699 - accuracy: 0.9874 - val_loss: 0.0900 - val_accuracy: 1.0000 - 10s/epoch - 102ms/step
Epoch 4/20
100/100 - 9s - loss: 0.0912 - accuracy: 0.9922 - val_loss: 0.0352 - val_accuracy: 1.0000 - 9s/epoch - 86ms/step
Epoch 5/20
100/100 - 9s - loss: 0.0470 - accuracy: 0.9954 - val_loss: 0.0286 - val_accuracy: 0.9984 - 9s/epoch - 87ms/step
Epoch 6/20
100/100 - 9s - loss: 0.0270 - accuracy: 0.9970 - val_loss: 0.0081 - val_accuracy: 1.0000 - 9s/epoch - 88ms/step
Epoch 7/20
100/100 - 9s - loss: 0.0168 - accuracy: 0.9982 - val_loss: 0.0031 - val_accuracy: 1.0000 - 9s/epoch - 89ms/step
Epoch 8/20
100/100 - 9s - loss: 0.0151 - accuracy: 0.9960 - val_loss: 0.0028 - val_accuracy: 1.0000 - 9s/epoch - 90ms/step
Epoch 9/20